In [ ]:
# IMDB & CNN
# *Dataset에 대해 추가 조사 필요
# - 실제 데이터에 대한 상세 기술
# - 예측하고자 하는 방법/방향 기술 : batch size를 64, hidden dimension을 250으로 CNN을 통해 테스트해서 accuracy를 높이고자 했다.
# - 학습을 위해 데이터가 어떻게 가공/처리되었는지 기술
# - 사용된 모델의 특징에 대한 기술
# - 생성된 모델의 model.summary() 출력
# - epoch은 최소 25회 이상 진행

In [ ]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, Activation
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing import sequence

In [ ]:
MAX_features = 5000
MAX_len = 400
Batch_size = 64
Embedding_dimension = 120
Filters = 250
Kernel_size = 3
Hidden_dimension = 250
Epoch = 25

In [ ]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=MAX_features)

In [ ]:
x_train = sequence.pad_sequences(x_train, maxlen=MAX_len)
x_test = sequence.pad_sequences(x_test, maxlen=MAX_len)
print(x_train[0])
print(y_train[0])

In [ ]:
model = Sequential()
model.add(Embedding(MAX_features,
                    Embedding_dimension,
                    input_length=MAX_len))
model.add(Dropout(0.2))

In [ ]:
# Conv1D는 간단한 데이터 학습 예측에 적합한 컨볼루션 레이어이다.
# 특정 부분만 추출 가능(전체 불가능)
model.add(Conv1D(Filters,
                 Kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))

In [ ]:
# 가장 큰 벡터값만 추출해주는 풀링 함수이다.
model.add(GlobalMaxPooling1D())

In [ ]:
# 히든 레이어는 입력층과 결과층 사이에 존재하는 학습에 중대한 영향을 끼치는 계층이다.
model.add(Dense(Hidden_dimension))

# Dropout -> 네트워크의 일부를 생략 -> 생략한 네트워크는 학습에 영향을 안끼침
# 특정 뉴런의 bias나 weight가 큰 값을 가지면
# 이 뉴련의 영향이 커지게 되면서 다른 뉴런들의 학습 속도가 느려지거나 악영향을 끼칠 수 있다.
# 이런 특정 뉴런의 영향을 받지 않기 떄문에 co-adaptation이 되는 것을 방지 할 수 있다.
model.add(Dropout(0.2))

# sigmoid 함수는 backpropagation 수행할 때 각 계층들을 지나면서 gradient를 계속 곱하게
# 되어서 gradient는 0으로 수렴한다. 그래서 계층이 많아지면 동작하지 않을 수 있다.
# relu 함수는 input value가 0보다 작으면 0 / 0보다 크면 입력값을 보존한다.
model.add(Activation('relu'))

In [ ]:
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train,
          batch_size=Batch_size,
          epochs=Epoch,
          validation_data=(x_test, y_test))
history_dict = history.history
history_dict.keys()

In [ ]:
model.summary()

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy : %.2f%%" % (scores[1]*100))

In [ ]:
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='loss')
plt.plot(epochs, val_loss, 'b', label='val_loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()